In [17]:
import os
import pandas as pd
import sys
import time
import datetime
import time
from collections import defaultdict
import numpy as np

## 인자값 입력
### 1) 충/방전 분류 +환경데이터 병합 데이터파일이 있는 디렉토리 경로

In [18]:
_file_input_path = './data'

In [19]:
# 두 형식을 모두 지원하도록 동적 처리
def to_seconds(date_str):
    if '.' in date_str:
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S.%f")
    elif len(date_str) >= 23:
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S:%f")
    else:
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return date_obj.timestamp()


def printProgressBar(iteration, total, prefix = 'Progress', suffix = 'Complete',\
                      decimals = 1, length = 50, fill = '█'): 
    # 작업의 진행상황을 표시
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' %(prefix, bar, percent, suffix), end='\r')
    sys.stdout.flush()
    if iteration == total:
        print()


def recursive_search_dir(_nowDir, _filelist, _form = 'csv'):
    dir_list = []  # 현재 디렉토리의 서브디렉토리가 담길 list
    if _nowDir[-1] == '/':
        _nowDir = _nowDir[0:-1]
    f_list = os.listdir(_nowDir)
    for fname in f_list:
        if os.path.isdir(_nowDir + "/" + fname):
            dir_list.append(_nowDir + "/" + fname)
        elif os.path.isfile(_nowDir + "/" + fname):
            file_extension = os.path.splitext(fname)[1]
            if file_extension == '.' + _form.lower() or file_extension == '.' + _form.upper():  # csv
                _filelist.append(_nowDir + "/" + fname)

    for toDir in dir_list:
        recursive_search_dir(toDir, _filelist, _form)

In [22]:
file_list = []
recursive_search_dir(_file_input_path, file_list)

charge_statistic_df = pd.DataFrame(columns=[
    "car_id", "car_type", "charge_type", "start_time", "end_time", "soh", "p_volt", "p_current",
    "cell_volt_diff", "soc_start", "soc_end", "soc_quan", "period", "charge_discharge_speed",
    "power_quan", "power_per_hour", "power_per_soc", "module_temp",
    "temperature", "mileage", "accum_mileage", "km_per_kWh", 'speed_0', 'speed_0_30', 'speed_30_60', 'speed_60_90', 'speed_90_180', "model_year", "model_month"
])

cnt = 1
t_cnt = len(file_list)
for csv_file in file_list:
    printProgressBar(cnt, t_cnt)
    # 차량id, 차종, 급속/완속/방전 타입, 구간 시작시간, 구간 끝 시간, soh 평균, 팩 전압 평균, 팩 전류 평균, 
    # 셀 최대/최소 전압차 평균, soc 시작 값, soc 끝 값, soc 충/방전 량, 구간 기간(초), 1시간당 SOC 충방전 속도,
    # 소모/충전 전력 량, 1시간당 소모/충전된 전력량, SOC 1%당 소모/충전된 전력 량, 모듈 1~4 평균 온도 평균,
    # 기온 평균, 구간 주행거리, 누적 주행거리, 전비, 연식(년), 연식(월)

    # car_id, car_type, charge_type, start_time, end_time, soh, p_volt , p_current, \
    # cell_volt_diff, soc_start, soc_end, soc_quan, period, charge_discharge_speed, \
    # power_quan, power_per_hour, power_per_soc, module_temp, \
    # temperature, mileage, accum_mileage, km_per_kWh, model_year, model_month
    
    usecol_list = ['COLL_DT', 'CAR_TYPE', 'B_SOH', 'B_PACK_VOLT', 'B_PACK_CURRENT', 'B_MAX_CELL_VOLT', 'B_MIN_CELL_VOLT', 'B_SOC', \
                        'B_ACCUM_DISCHARG_POWER_QUAN', 'B_ACCUM_CHARG_POWER_QUAN', 'B_MODUL_1_TEMP', 'B_MODUL_2_TEMP',\
                               'V_CAR_SPEED', 'B_MODUL_3_TEMP', 'B_MODUL_4_TEMP', 'TEMPERATURE', 'C_MILEAGE']
    available_cols = pd.read_csv(csv_file, nrows=0).columns  # 첫 줄(헤더)만 읽음
    # 대소문자 무시하고 비교하기 위해 모두 대문자로 변환
    available_cols_upper = [col.upper() for col in available_cols]

    # 교집합 구하기
    valid_usecols = [col for col in usecol_list if col in available_cols_upper]

    # 실제 읽을 때는 원래 칼럼 이름 사용 (소문자/대문자 유지)
    valid_usecols_original = [available_cols[i] for i, col in enumerate(available_cols_upper) if col in valid_usecols]

    # 유효한 칼럼만 읽기
    df = pd.read_csv(csv_file, low_memory=False, usecols=valid_usecols_original)
    if len(df) < 10:
        continue
    
    # 모든 컬럼명을 대문자로 변환 (년도별로 소문자 대문자 섞여있음)
    df.columns = df.columns.str.upper()
    df.sort_values(['COLL_DT'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    # 사전 전처리 (첫, 끝부분의 각 index간 시간차이가 1분이상 나면 잘라버림, 이전/다음 구간의 데이터가 포항된 경우가 있어서)
    total_rows = len(df)
    cutoff = int(total_rows * 0.05)
    cutoff_1 = -1
    cutoff_2 = total_rows
    # **첫 5% 구간에서 탐색**
    for idx in range(cutoff,0, -1):
        time_diff = (to_seconds(df['COLL_DT'].iloc[idx]) - to_seconds(df['COLL_DT'].iloc[idx-1])) / 60
        if time_diff > 1:  # 1분 이상 차이나면 해당 인덱스까지 제거
            cutoff_1 = idx-1
            break

    # **끝 5% 구간에서 탐색**
    for idx in range(total_rows-cutoff,total_rows):
        time_diff = (to_seconds(df['COLL_DT'].iloc[idx]) - to_seconds(df['COLL_DT'].iloc[idx-1])) / 60
        if time_diff > 1:  # 1분 이상 차이나면 해당 인덱스까지 제거
            cutoff_2 = idx
            break
    # 조건에 따라 인덱스 제외
    df = df[(df.index > cutoff_1) & (df.index < cutoff_2)]
    df.reset_index(drop=True, inplace=True)
    
    # 차량id
    car_id = csv_file.split('/')[-1]
    car_id = car_id.split('_')[0]
    # 차종
    car_type = df.iloc[0]['CAR_TYPE']
    car_type = car_type.upper()
    # 급속/완속/방전 구분필드
    charge_type = csv_file.split('/')[2]
    if charge_type == 'discharge':
        charge_type = 'discharge'
    else:
        if 'fast' in charge_type:
            charge_type = 'fast_charge'
        else:
            charge_type = 'slow_charge'
    # 구간 시작, 끝시간
    start_time = df.iloc[0]['COLL_DT']
    end_time = df.iloc[-1]['COLL_DT']
    # soh 값
    soh = df['B_SOH'].mean()
    # 팩 전압 평균
    p_volt = df['B_PACK_VOLT'].mean()
    # 팩 전류 평균
    p_current = df['B_PACK_CURRENT'].mean()
    # 셀 전압차 평균 b_max_cell_volt, b_min_cell_volt
    cell_volt_diff = (df['B_MAX_CELL_VOLT'] - df['B_MIN_CELL_VOLT']).mean()
    # soc 시작, 끝 값
    soc_start = df.iloc[0]['B_SOC']
    soc_end = df.iloc[-1]['B_SOC']
    # soc 충방전량
    if charge_type == 'discharge':
        soc_quan = soc_start - soc_end
    else:
        soc_quan = soc_end - soc_start
    # 구간 기간 (초)
    period = to_seconds(end_time) - to_seconds(start_time)
    # 충/방전 속도 (1시간당)
    try: charge_discharge_speed = soc_quan / period * 3600
    except: charge_discharge_speed = float('nan')
    # 소모/충전 전력 량
    if charge_type == 'discharge':
        power_quan = df.iloc[-1]['B_ACCUM_DISCHARG_POWER_QUAN'] - df.iloc[0]['B_ACCUM_DISCHARG_POWER_QUAN']
    else:
        power_quan = df.iloc[-1]['B_ACCUM_CHARG_POWER_QUAN'] - df.iloc[0]['B_ACCUM_CHARG_POWER_QUAN']
    # 1시간당 전력 소모/충전 량
    try: power_per_hour = power_quan / period * 3600
    except: power_per_hour = float('nan')
    # SOC 1% 당 소모/충전된 전력 량
    try: power_per_soc = power_quan / soc_quan
    except: power_per_soc = float('nan')
    # 모듈 1~4 온도 평균
    module_temp = df[['B_MODUL_1_TEMP', 'B_MODUL_2_TEMP', 'B_MODUL_3_TEMP', 'B_MODUL_4_TEMP']].values.mean()
    # 기온
    temperature = df['TEMPERATURE'].mean()
    # 구간 주행거리
    mileage_s = df.iloc[0]['C_MILEAGE']
    mileage_e = df.iloc[-1]['C_MILEAGE']
    if pd.isna(mileage_s):
        mileage_s = df.iloc[1]['C_MILEAGE']
    if pd.isna(mileage_e):
        mileage_e = df.iloc[-2]['C_MILEAGE']
    mileage = mileage_e - mileage_s
    if mileage_e != 0 and mileage_s != 0:
        mileage = mileage_e - mileage_s
        if mileage >= 10000: # 1) 충전 없이 10,000km 이상 방전한 경우는 오류로 판단, 2) 방전 중 주행거리가 0 이하일 경우 오류로 판단
            mileage = float('nan')
        if charge_type == 'discharge' and mileage <= 0:
            mileage = float('nan')
        if charge_type != 'discharge' and mileage != 0:
            mileage = float('nan')
    else:
        mileage = float('nan')
    # 누적 주행거리
    try: accum_mileage = mileage_s + mileage
    except: accum_mileage = float('nan')
    # 전비
    try: km_per_kWh = mileage / power_quan
    except: km_per_kWh = float('nan')
    
    if charge_type == 'discharge':
        SPEED_0 = df[df['V_CAR_SPEED'] == 0].shape[0]
        SPEED_0_30 = df[(df['V_CAR_SPEED'] > 0) & (df['V_CAR_SPEED'] <= 30)].shape[0]
        SPEED_30_60 = df[(df['V_CAR_SPEED'] > 30) & (df['V_CAR_SPEED'] <= 60)].shape[0]
        SPEED_60_90 = df[(df['V_CAR_SPEED'] > 60) & (df['V_CAR_SPEED'] <= 90)].shape[0]
        SPEED_90_180 = df[df['V_CAR_SPEED'] > 90].shape[0]
    else:
        SPEED_0 = float('nan')
        SPEED_0_30 = float('nan')
        SPEED_30_60 = float('nan')
        SPEED_60_90 = float('nan')
        SPEED_90_180 = float('nan')
        
    
    model_year = float('nan')
    model_month = float('nan')
    
    append_row = [car_id, car_type, charge_type, start_time, end_time, soh, p_volt , p_current, \
                cell_volt_diff, soc_start, soc_end, soc_quan, period, charge_discharge_speed, \
                    power_quan, power_per_hour, power_per_soc, module_temp, \
                        temperature, mileage, accum_mileage, km_per_kWh, SPEED_0, SPEED_0_30, SPEED_30_60, SPEED_60_90, SPEED_90_180, model_year, model_month]

    append_row_df = pd.DataFrame([append_row], columns=[
    "car_id", "car_type", "charge_type", "start_time", "end_time", "soh", "p_volt", "p_current",
    "cell_volt_diff", "soc_start", "soc_end", "soc_quan", "period", "charge_discharge_speed",
    "power_quan", "power_per_hour", "power_per_soc", "module_temp",
    "temperature", "mileage", "accum_mileage", "km_per_kWh", 'speed_0', 'speed_0_30', 'speed_30_60', 'speed_60_90', 'speed_90_180', "model_year", "model_month"
])
    charge_statistic_df = pd.concat([charge_statistic_df, append_row_df])
    cnt+=1
charge_statistic_df = charge_statistic_df.sort_values(by=['start_time'])
charge_statistic_df.reset_index(drop=True, inplace=True)

charge_statistic_df

/tmp/ipykernel_581310/1241144074.py:176: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  charge_statistic_df = pd.concat([charge_statistic_df, append_row_df])


Progress |██████████████████████████████████████████████████| 100.0% Complete


,car_id,car_type,charge_type,start_time,end_time,soh,p_volt,p_current,cell_volt_diff,soc_start,...,mileage,accum_mileage,km_per_kWh,speed_0,speed_0_30,speed_30_60,speed_60_90,speed_90_180,model_year,model_month
0,M2210703926,IONIQ5,fast_charge,2022-01-02 17:21:23.000,2022-01-02 18:00:58.000,100.000000,720.501020,-70.803571,0.011633,54.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1200607457,IONIQ5,slow_charge,2022-01-05 19:28:00.000,2022-01-05 20:57:53.000,100.000000,664.299630,-34.756296,0.005370,36.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M2210703914,IONIQ5,discharge,2022-01-05 19:31:31.000,2022-01-05 20:54:29.000,99.783550,638.507991,16.628726,0.015853,50.5,...,118.0,8419.0,4.836066,51.0,21.0,5.0,78.0,305.0,NaN,NaN
3,M1200607457,IONIQ5,slow_charge,2022-01-05 23:58:09.000,2022-01-06 02:19:41.000,100.000000,725.809198,-33.879599,0.007689,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M2210703775,IONIQ5,fast_charge,2022-01-10 07:54:25.000,2022-01-10 08:04:16.000,100.000000,658.495000,-98.700000,0.023667,9.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M2210703926,IONIQ5,discharge,2022-01-10 09:44:23.000,2022-01-10 11:14:16.000,99.814815,668.780000,-3.445741,0.006407,60.0,...,36.0,2440.0,3.673469,78.0,288.0,133.0,25.0,0.0,NaN,NaN
6,M2210703914,IONIQ5,discharge,2022-01-10 10:29:23.000,2022-01-10 11:20:44.000,100.000000,699.547249,14.888673,0.008091,83.0,...,70.0,8672.0,4.191617,16.0,36.0,30.0,49.0,177.0,NaN,NaN
7,M2210703914,IONIQ5,fast_charge,2022-01-10 12:08:46.000,2022-01-10 12:14:27.000,100.000000,733.165714,-144.482857,0.018857,67.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,M2210703926,IONIQ5,discharge,2022-01-10 21:06:08.000,2022-01-10 21:53:09.000,100.000000,650.711661,2.400707,0.005936,43.5,...,33.0,2481.0,3.928571,22.0,71.0,113.0,71.0,0.0,NaN,NaN
9,M2210703926,IONIQ5,fast_charge,2022-01-10 21:57:43.000,2022-01-10 22:13:04.000,100.000000,682.666667,-89.620430,0.018280,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
